# Modelling the Cobot

Since the model provided was not complete w.r.t. several attributes, a re-modelling of the URDF was required.

Required adaptions:

1. Position of elements:
   the original URDF model contained transformations (rotations and translations) within the joints as well as within the link tags. This made the entire manipulation difficult. Hence, the first task was to re-define the link / joint positions based on the following rule: transformations have to be handled only within the joints. This increases readability and simplifies the remaining tasks.
   
   <img src="img/inertial_urdf.png" alt="inertial_urdf" width="300"/>
   
   (source: [ROS Wiki](http://wiki.ros.org/urdf/XML/link))

   This task was straightforward and does nor require longer explanations. We used the [Phobos Blender Plugin](https://github.com/dfki-ric/phobos) for the manual adaptions of the transformations.
2. Modelling physical attributes:
   neither the mass nor the inertia values were present within the original URDF. The attempts to derive the values for each robot element are described in the following.
3. The URDF did not define any collision tags. We have reused the visual meshes (with reduced faces to derive realistic collisions for each segment).

# Robot Overview

The cobot model defines 7 segments, 2 fingers, one TCP element (representing the end-effector) as well as an axis element and a corresponding mount. Segments, fingers, the axis, the carriage and the TCP represent "links" in URDF.

<img src="img/cobot_full.png" alt="cobot_full" width="400"/>


# Deriving Physical Characteristics

We will focus on the mass and the inertia values for each element in the following.

## Estimating the Mass of Segments

Since we do not have any information on the weight of each segment, we will estimate it using their volumes.

### Volume Estimation

Since our meshes are quite detailed, we can use them to identify the volume of the segments with 3D computer graphic software. Our preferred choices are [Blender](https://www.blender.org) and [MeshLab](https://www.meshlab.net). Both offer extensive support for mesh manipulation.<br/>

For cross checks, we will estimate the volume by approximating the mesh with basic shapes: cuboids and cylinders.
All we need is the size of the object in each dimension (as a bounding box or a radius with the height; both shown in Blender / MeshLab) and the corresponding formula for the volume calculation:<br/>

Cuboid: $V = a \cdot b \cdot c$, with $a,b,c$ representing the size of the object in $x,y,z$ direction<br/>

Cylinder: $V = \pi \cdot r^2 \cdot h$, with $r$ representing radius of the cylinder and $h$ the height
<br/>

In [2]:
import math as m

def get_volume_of_cylinder(r, z):
    return m.pi * m.pow(r,2) * z

def get_volume_of_cuboid(x, y, z):
    return x * y * z

#### Volume Estimation in Blender

```
1. import the mesh in blender
3. in object properties toolbox: scale X,Y,Z by 0.001 (to get meters as units)
3. Edit -> Preferences -> Addons -> activate 3D Print Toolbox
4. click on item, volume is shown in the 3d Print Toolbox menu
```

#### Volume Estimation in MeshLab

Since we do not solely rely on Blender, we will also use MeshLab. Here, we need to first make the mesh "watertight" (interpreting the mesh as one closed surface): 
```
1. import mesh
2. Filters -> Remeshing, Simplification and Reconstruction -> Surface Reconstruction: Screened Poisson (default options)
```
Applying the screened poisson (a commonly used reconstruction method for complex shapes) will approximate its entire shape and will allow to calculate the geometric measures of the object.
<br/>
(It does sound a bit odd to reconstruct a mesh shape that is looks complete and functional. The reason behind seem to be missing information in the STL files that represent the meshes.)
<br/>
To get the geometric data from MeshLab, apply the following operations.
```
3. Filters -> Normals, Curvatures and Orientation -> Transform: Scale, Normalize -> scale X,Y,Z by 0.001
4. Filters -> Quality, Measures and Computation -> Compute Geometric Measures
(read the values either from the terminal or from the log window in MeshLab)
```

Note: MeshLab will also print the inertia tensor values that we are calculating here. Unfortunately these do not seem to align with out manual calculation and they are not working with gazebo (it complains about the values being "invalid"). They are also calculated based on the principal axis (which is not always correct in our case - refer below).<br/>

### Deriving Mass from Volume 

Once we have the volume, we derive the mass using 
$$
m = \rho \cdot V
$$ <br/>
where $\rho$ represents the density and $V$ the volume. <br/>

For the density, we need to define a substance that fills the segment. Since it's usually a bunch of cables, aluminum, plastic, copper and air, we need to approximate these types all together. One idea is to expect the segment to be filled with water. Although this seems a bit overly simplified, it serves as a good estimation (and after all, what we need in the simulation are not actual values but ratios of physical attributes for elements w.r.t. each other).<br/>
The density of water is defined as 
$$
1000\frac{kg}{m^3},
$$

implying that if an object has a volume of $0.001m^3$ and is filled with water, its mass results in $m = 1000\frac{kg}{m^3} \cdot 0.001m^3 = 1.0kg$.<br/>

In [3]:
def get_mass_of_object(d, V):
    return d * V

## Moments of Inertia

When coping with simulation, we need to consider physical characteristics of objects such as their mass and their inertia; which using a simplified definition describes an object's "resistance" when rotational forces are applied to it. This resistance increases more than proportional with the size of the object and has a significant impact on the simulation.<br/>

**Simplification of inertia**: the higher the inertia, the more torque needs to be applied in order to rotate an object.

<br/>
Since our goal is to enable a realistic simulation, we need to provide the inertia values for each component in our model (each segment / link of the robot).

Note: inertia only applies to **revolute** joints. Hence, the elements **lin_axis**, **lin_axis_carriage**, **joint_0** and the **gripper** (being attached to prismatic joints) do not require them. Gazebo and other simulations still require each element to be assigned to an inertia tensor, however. The background is that although rotational forces are not applied to prismatic joints, their inertias are relevant for a system-wide simulation.


### Inertia Tensors

... describe the inertia in each direction ($x,y,z$). In order to simplify the derivation of the inertia values, we will approximate our robot elements with two basic shapes: cuboids and cylinders.

![cuboid.png](img/cuboid.png) ![cylinder.png](img/cylinder.png)

The inertia tensors of solid cuboids and cylinders that are rotated around the principal axis (the center of each axis) are shown in the following.

#### Cuboid

$I = \frac{m}{12} \cdot \begin{pmatrix}  y^2 + z^2 & 0 &0 \\
                                         0 & x^2 + z^2  & 0 \\
                                         0 & 0 & x^2 + y^2 \\
 \end{pmatrix}$<br/>

where $m$ represents the mass of the cuboid and $x,y,z$ the lengths of the object in their corresponding axis.


In [ ]:
import numpy as np
def get_inertia_tensor_of_cuboid(mass, x, y, z):
    inertia_xx = (mass / 12.0) * (y**2 + z**2)
    inertia_yy = (mass / 12.0) * (x**2 + z**2)
    inertia_zz = (mass / 12.0) * (x**2 + y**2)
    return np.array([[inertia_xx, 0, 0], [0, inertia_yy, 0], [0, 0, inertia_zz]])

#### Cylinder

$I = \frac{m}{12} \cdot \begin{pmatrix}  3 \cdot r^2 + z^2  & 0 &0 \\
                                         0 & 3 \cdot r^2 + z^2 & 0 \\
                                         0 & 0 & 3 \cdot m \cdot r^2 \\
 \end{pmatrix}$<br/>

 where $m$ represents the mass of an object, $r$ the radius and $z$ the length of the cylinder in $z$ direction.


In [ ]:
import numpy as np
def get_inertia_tensor_of_cylinder(m, r, z):
    inertia_xx = inertia_yy = (m / 12.0) * (3*r**2 + z**2)
    inertia_zz = (m / 12.0) * (6*r**2)
    return np.array([[inertia_xx, 0, 0], [0, inertia_yy,0], [0,0, inertia_zz]])

### Custom Point of Rotation in Inertia Tensor

In some cases we will need to adjust the rotation (not all elements are rotated around their center in our case). For this purpose, we will use the "parallel axes theorem" (also referred to as "Steiner's theorem") that allows a simple adaption to the center of rotation.

$ I=I_{c}+ m \cdot \begin{pmatrix} d_y^2 + d_z^2  & -d_x \cdot d_y & -d_x \cdot d_z \\
                 -d_x \cdot d_y & d_x^2 + d_z^2 & -d_y \cdot d_z \\
                 -d_x \cdot d_z  &  -d_y \cdot d_z & d_x^2 + d_y^2 \\
 \end{pmatrix}$

where $I_{c}$ represents the inertia tensor around the principal axis and $d_x, d_y, d_z$ the displacement in the corresponding direction from the center.

![rotation_axis](img/rotation_axis.png)

In [6]:
def calc_steiner_offset(mass, dx, dy, dz):
    return mass * np.array([[dy**2 + dz**2, -dx*dy, -dx*dz], [-dx*dy, dx**2 + dz**2, -dy*dz], [-dx*dz ,-dy*dz, dx**2 + dy**2]])

### Center of Mass

The center of mass refers to the object center point. In URDF it is set to the joint's origin by default. This implies that the rotation of the element is set to the principle axis ($ x = y = z = 0$); which is often not correct.
An example is shown in the following: if the center of mass is identical with the joint's origin, the element's center is expected to be positioned it's principle axis; to model its real center, we will shift the center of mass. The values for the shift are the same as for the parallel axes theorem. The center of mass is therefore closely related to the rotation point.

![center_of_mass](img/center_of_mass.png)

### Representation in URDF
In URDF these values are defined within the ```inertial``` element that defines the sub-elements ```mass```, ```origin``` and ```inertia```; where ```origin``` describes the **center of mass** and ```inertia``` the inertia tensor.

```
<inertial>
 <mass value="7.3" />
 <origin rpy="0.0 0.0 0.0" xyz="0.0 0.0 0.0" />
 <inertia ixx="0.0" iyy="0.0" izz="0.0" ixy="0.0" ixz="0.0" iyz="0.0" />
</inertial>
```

### The Cuboid Inertia Tensor

The following paragraphs represents a short proof on the cuboid inertia tensor. 

Background: we initially used the the cuboid inertia tensor from Wikipedia and after spending several hours trying to understand why my inertia values are incorrect, we figured out that the Wikipedia entry contained a typo and instead of just using a different source for the tensor, we made an attempt to prove the inertia tensor ourself.

In general, the inertia tensor for solid $3$-dimensional objects rotating around their principal axis is defined as
$$
I_{i,i} = \int_{V} \rho \cdot r_{i,i}^2 dV
$$
where $i \in [1,2,3]$, representing the axis directions, $V$ the volume of the object, $\rho$ its density and $r^2$ the squared distance to any orthogonal point. For a cube in $x$ direction we have 
$$
I_{x,x} = \int_{V} \rho \cdot (y^2 + z^2) dV
$$


![cuboid.png](img/cuboid_center.png)

Since we consider only a rotation around the principle axis, we ignore all elements $j \neq i$ and hence use a diagnonal matrix as the inertia tensor:
$$
I_{cm} = \begin{pmatrix}  I_{xx} & 0 &0 \\
                          0 &  I_{yy}  & 0 \\
                           0 & 0 &  I_{zz} \\
 \end{pmatrix}
$$

where 
$$
I_{x,x} = \int_{-\frac{a}{2}}^{\frac{a}{2}}\int_{-\frac{b}{2}}^{\frac{b}{2}}\int_{-\frac{c}{2}}^{\frac{c}{2}} \rho (y^2 + z^2) dxdydz
$$
$$
I_{y,y} = \int_{-\frac{a}{2}}^{\frac{a}{2}}\int_{-\frac{b}{2}}^{\frac{b}{2}}\int_{-\frac{c}{2}}^{\frac{c}{2}} \rho (x^2 + z^2) dxdydz
$$
$$
I_{z,z} = \int_{-\frac{a}{2}}^{\frac{a}{2}}\int_{-\frac{b}{2}}^{\frac{b}{2}}\int_{-\frac{c}{2}}^{\frac{c}{2}} \rho (x^2 + y^2) dxdydz
$$
with $a$ representing the size of the cube in $x$, $b$ in $y$ and $c$ in $z$ direction. We use $\pm\frac{a}{2}$,  $\pm \frac{b}{2}$ and $\pm \frac{c}{2}$ as the ingetration limits since we place the cube's center point at the origin of its frame of reference $x = y = z = 0$ (refer image above).

For $I_{xx}$ we have

\begin{align}
I_{x,x} = &  \int_{-\frac{a}{2}}^{\frac{a}{2}}\int_{-\frac{b}{2}}^{\frac{b}{2}}\int_{-\frac{c}{2}}^{\frac{c}{2}} \rho (y^2 + z^2) dxdydz \\
 & = \rho (\int_{-\frac{a}{2}}^{\frac{a}{2}}\int_{-\frac{b}{2}}^{\frac{b}{2}}\int_{-\frac{c}{2}}^{\frac{c}{2}} (y^2 + z^2) dxdydz) \\
 &\stackrel{\text{Fubini}}{=} \rho \biggl( \int_{-\frac{a}{2}}^{\frac{a}{2}} dx \cdot \int_{-\frac{b}{2}}^{\frac{b}{2}} y^2 dy + \int_{-\frac{a}{2}}^{\frac{a}{2}} dx \cdot \int_{-\frac{c}{2}}^{\frac{c}{2}}  z^2 dz \biggr) \\
 &\stackrel{\text{}}{=} \rho \left( x |_{-\frac{a}{2}}^{\frac{a}{2}} \cdot \frac{1}{3} y^3 |_{-\frac{b}{2}}^{\frac{b}{2}} + x |_{-\frac{a}{2}}^{\frac{a}{2}} \cdot \frac{1}{3} z^3 |_{-\frac{a}{2}}^{\frac{c}{2}} \right)\\
 = & \rho \left( (\frac{a}{2} - (-\frac{a}{2})) \cdot \frac{1}{3} \cdot \left(\frac{b}{2}^3 - (-\frac{b}{2}^3) \right) + (\frac{a}{2} - (-\frac{a}{2})) \cdot \left(\frac{c}{2}^3 - (-\frac{c}{2})^3) \right)  \right)\\
  = & \rho \left( a \cdot \frac{1}{3} \cdot 2 \cdot (\frac{b}{2})^3 + a \cdot \frac{1}{3} \cdot2 \cdot (\frac{c}{2})^3  \right)\\
  = & \rho \cdot a \left(\frac{1}{3} \cdot 2 \cdot (\frac{b}{2})^3 + \frac{1}{3} \cdot2 \cdot (\frac{c}{2})^3  \right)\\
  = & \rho \cdot a \left(\frac{1}{3} \cdot \frac{2b^3}{6} + \frac{1}{3} \cdot \frac{2c^3}{6}  \right)\\
  = & \rho \cdot a \left(\frac{2b^3}{24} + \frac{2c^3}{24}  \right)\\
  = & \rho \cdot a \left(\frac{b^3}{12} + \frac{c^3}{12}  \right)\\
  = & \rho \cdot a \cdot b \cdot c \left(\frac{b^2}{12} + \frac{c^2}{12}  \right)\\
  = & \frac{\rho \cdot a \cdot b \cdot c}{12} \left(b^2 +c^2 \right)
\end{align}

Since $ \rho \cdot a \cdot b \cdot c$ represents the mass $m$ and since the equation is straigthforward for $I_{yy}$ as well as $I_{zz}$ the result is the initial tensor defined above.

$I = \frac{m}{12} \cdot \begin{pmatrix}  b^2 + c^2 & 0 &0 \\
                                         0 & a^2 + c^2  & 0 \\
                                         0 & 0 & a^2 + b^2 \\
 \end{pmatrix}$<br/>

with $a,b,c$ representing the length of the object in $x,y,z$ respectively.

An important observation here is that with an increased size of an object, let's say in scaled in each dimension, the inertia values increase non-linear.<br/>

Example:
for a cube of size $a=b=c=2$, and mass $12kg$ the inertia tensor is

$I_{cube} =  \begin{pmatrix}  8 & 0 &0 \\
                              0 & 8  & 0 \\
                              0 & 0 & 8 \\
 \end{pmatrix}$<br/>

 while for a cube of size $a=b=c=4$ with the same mass the inertia tensor is

 $I_{cube} =  \begin{pmatrix} 32 & 0 &0 \\
                              0 & 32 & 0 \\
                              0 & 0 & 32 \\
 \end{pmatrix}$<br/>

This implies that the size of an object has a significant impact on its resulting inertia values.

## Segment 1

<img src="img/segment_1.png" alt="segment.png" width="200"/> <img src="img/segment_1_r.png" alt="segment_1_r.png" width="200"/>

### Characteristics
<br/>

**Bounding box**<br/>
```
x = 0.189 
y = 0.189 
z = 0.312
```

**Frame of reference**<br/>
Origin is located at the top => the element's bottom is located at $z=-0.312m$  => $z:= -z$

**Approx. geometric shape**<br/>
We will approximate the element's shape with a cylinder of radius $r=0.076m$



### Deriving Center of Mass

***In $z$ direction: $d_z$***

For the URDF, we need to designate the center of mass from the joint's origin; in the following referred to as the offset in $z$-axis: $d_z$.

What we know is

1. size of segment 1 along the $z$ axis is defined as $0.312$m. We will estimate the center of $z$ with $z_{center} = \frac{0.312m}{2} = 0.156m$
2. the element's max value for depth is $z=-0.312m$. This implies using the negative sign for the offset.

Hence,<br/>
$d_z = z_{center} = -\frac{0.312m}{2} = -0.156m$

<br/>
Since segment 1 is a simple shape, we do not need to consider offsets for the $x$ and the $y$ direction. Our position offset regarding the center of mass is therefore simple.

```
<inertial>
...
  <origin rpy="0.0 0.0 0.0" xyz="0.0 0.0 -0.156" />
...
</inertial>
```

### Deriving the Volume / Mass

Blender output is $0.0073m^3$. Cross check with MeshLab yields a similar value by applying the "Convex Hull" remesh filter while calculating the volume from a cylinder based on the radius set above yields $0.00566m^3$ (refer calculation below). We will use the blender output for the volume and hence set $V = 0.0073$

In [7]:
r=0.076; z=0.312
print(get_volume_of_cylinder(r,z))

0.005661501820146009


Next we derive the mass from the volume using $m = \rho \cdot V$, with $\rho = 1000\frac{kg}{m^3}$ (water).

This yields: $m = 1000\frac{kg}{m^3} \cdot 0.0073m^3 = 7.3kg$.<br/>

We will use $7.3kg$ as the mass of segment 1.

In [8]:
rho = 1000; V = 0.0073
print(str(get_mass_of_object(rho, V))+"kg")

7.3kg


### Inertia Tensor

We know that the rotation for this element is around the principle axis (center of each axis) and since it looks very similar to a cylinder, we use inertia tensor of a cylinder.<br/>

In [9]:
mass = 7.3; r = 0.076; z = 0.312;
print(get_inertia_tensor_of_cylinder(mass,r,z))

[[0.0697588 0.        0.       ]
 [0.        0.0697588 0.       ]
 [0.        0.        0.0210824]]


<br/>

```inertial``` attributes for segment 1:

```
<inertial>
 <mass value="7.3" />
 <origin rpy="0.0 0.0 0.0" xyz="0.0 0.0 -0.156" />
 <inertia ixx="0.0697588" iyy="0.0697588" izz="0.0210824" ixy="0.0" ixz="0.0" iyz="0.0" />
</inertial>
```

## Segment 2

![segment_2](img/segment_2.png)

### Characteristics


| Bounding box| Frame of Reference | Geometric Shape | Center of Mass | V Blender | V MeshLab | V Simple | Mass |
| -------- | ------- | ------- | ------- | ------- | ------- | ------- | ------- |
| x = 0.155| Same as joint | cuboid | $d_z = z_{center} = \frac{0.238}{2} = 0.119$ | <b>$0.005m^3$<b/> | $0.0049m^3$ |  $0.00686m^3$ | $5kg$ |
| y = 0.238 | | | | | | | |
| z = 0.186 | | | | | | | |

### Inertia Tensor

We know that the rotation for this element is around the principle axis (center of each axis) in $z$-direction. Hence, we apply the inertia tensor of a cuboid without modification.<br/>

In [10]:
mass=5.0; x=0.155; y=0.238; z=0.186
print(get_inertia_tensor_of_cuboid(mass,x,y,z))

[[0.03801667 0.         0.        ]
 [0.         0.02442542 0.        ]
 [0.         0.         0.03361208]]


```inertial``` information for segment 1:

```
<inertial>
 <mass value="5.0" />
 <origin rpy="0.0 0.0 0.0" xyz="0.0 0.0 0.119" />
 <inertia ixx="0.03801667" iyy="0.02442542" izz="0.03361208" ixy="0.0" ixz="0.0" iyz="0.0" />
</inertial>
```

## Segment 3
![segment_3](img/segment_3_plain.png)

### Characteristics

| Bounding box| Frame of Reference | Geometric Shape | V Blender | V MeshLab | V Simple | Mass |
| -------- | ------- | ------- |  ------- |  ------- |  ------- |  ------- |
| x = 0.372 | Same as joint | cuboid | $0.0025m^3$ | $0.0023m^3$ | $0.005999616m^3$ | $2.5kg$ |
| y = 0.126 | | | | | | |
| z = 0.128 | | | | | | |


### Deriving Center of Mass


***In $x$ direction: $d_x$***

What we know is

1. size of segment 3 along the $x$ axis is defined as $0.372$m. The center of $x$ is therefore $x_{center} = \frac{0.372m}{2} = 0.186m$
2. the element's joint is located close to $x=0$. We will set the joint's position at $x=0.06m$ based on the mesh (refer image below) and refer to it as $x_{joint}$.
 
    ![joint_pos_segment_3.png](img/joint_pos_segment_3.png)

Hence,<br/>
$d_x = x_{center} -x_{joint} = \frac{0.372m}{2} - 0.06m = 0.126m$

![segment_3_dx.png](img/segment_3_dx.jpg)


***In $z$ direction: $d_z$***

We can see that the end part segment 3 is slightly displaced along its $z$-axis (refer image below). The deviation displayed in blender is $\approx 0.06m$. We will use this value for $d_z$

![segment_3_dz.png](img/segment_3_dz.png)

***In $y$ direction: $d_y$***
Not required.
<br/>
<br/>
<br/>

With $d_x$ and $d_z$ known, we set the offsets for the center of mass for segment 3 in the URDF as

```
<inertial>
...
  <origin rpy="0.0 0.0 0.0" xyz="0.126 0.0 0.06" />
...
</inertial>
```


### Inertia Tensor
We know that the rotation for this element is not around the principle axis (center of each axis) but at the beginning of the element in $x$ direction (representing an arm). This implies the usage of the parallel axis theorem for the calculation of the inertia tensor.


Similar to the previous elements, we will estimate the inertia of the element with a cuboid but change its center of mass accordingly.

In order to limit over estimation of the cuboid, we will use the following cube size for the inertia tensor (we are basically scaling down the cuboid used to approximate the segment):
```
x = 0.262
y = 0.1
z = 0.102
```


We first calculate inertia tensor centered at the origin.

In [12]:
mass = 2.5; x = 0.262; y = 0.1; z = 0.102
get_inertia_tensor_of_cuboid(mass, x, y, z)

array([[0.00425083, 0.        , 0.        ],
       [0.        , 0.01646833, 0.        ],
       [0.        , 0.        , 0.01638417]])

Next, we apply the parallel axis theorem defined as <br/>
$$
I=I_{cm}+md^{2}
$$ <br/>
where $I_{cm}$ represents the inertia tensor of the principal axis, $m$ the mass and $d$ the distance from the principal axis (refer definition above). Taking over the values for $d_x$, $d_y$ and $d_z$ from the center of mass, we receive our final inertia tensor for segment 3.

In [13]:
mass = 2.5; x = 0.262; y = 0.1; z = 0.102
dx = 0.126; dy = 0; dz = 0.06
get_inertia_tensor_of_cuboid(mass, x, y, z) + calc_steiner_offset(mass, dx, dy, dz)

array([[ 0.01325083,  0.        , -0.0189    ],
       [ 0.        ,  0.06515833,  0.        ],
       [-0.0189    ,  0.        ,  0.05607417]])

```inertial``` attributes for segment 3:
```
<inertial>
 <mass value="2.5" />
 <origin rpy="0.0 0.0 0.0" xyz="0.126 0.0 0.06" />
 <inertia ixx="0.01325083" ixy="0.0" ixz="-0.0189" iyy="0.06515833" iyz="0.0" izz="0.05607417" />
</inertial>
```

## Segment 4

![segment_4](img/segment_4.png)

### Characteristics


| Bounding box| Frame of Reference | Geometric Shape | Center of Mass | V Blender | V MeshLab | V Simple | Mass |
| -------- | ------- | ------- | -------- | ------- | ------- | ------- | ------- |
| x = 0.118 | $z:= -z$ | cylinder: $r=0.14/2$ | $z_{center} = -\frac{0.225}{2} = -0.1122$ | $0.0058m^3$ | $0.0025m^3$ | $0.0034m^3$ | $3.4kg$ |
| y = 0.142 | | | | | | | |
| z = 0.225 | | | | | | | |


### Inertia Tensor

In [14]:
m = 3.4; r = 0.07; z = 0.225;
print(get_inertia_tensor_of_cylinder(m,r,z))

[[0.01850875 0.         0.        ]
 [0.         0.01850875 0.        ]
 [0.         0.         0.00833   ]]


```inertial``` attributes for segment 4:
```
<inertial>
 <mass value="3.4" />
 <origin rpy="0.0 0.0 0.0" xyz="0.0 -0.112 0.0" />
 <inertia ixx="0.01850875" ixy="0.0" ixz="0.0" iyy="0.01850875" iyz="0.0" izz="0.00833" />
</inertial>
```

## Segment 5

![segment_5](img/segment_5.png)
### Characteristics

| Bounding box| Frame of Reference | Geometric Shape | Center of Mass | V Blender | V MeshLab | V Simple | Mass |
| -------- | ------- | ------- | -------- | ------- | ------- | ------- | ------- |
| x = 0.123 | Same as joint | cylinder: $r=z_{center}$ | $z_{center} = \frac{0.143}{2} = 0.071$ | $0.0011m^3$ | $0.00118m^3$ | $0.0022m^3$ | $1.1kg$ |
| y = 0.108 | | | | | | | |
| z = 0.143 | | | | | | | |

### Inertia Tensor

In [15]:
mass = 1.1; r = 0.071; z = 0.143;
print(get_inertia_tensor_of_cylinder(mass,r,z))

[[0.00326077 0.         0.        ]
 [0.         0.00326077 0.        ]
 [0.         0.         0.00277255]]


```inertial``` attributes for segment 5:
```
<inertial>
 <mass value="1.1" />
 <origin rpy="0.0 0.0 0.0" xyz="0.0 0.0 0.071" />
 <inertia ixx="0.00326077" iyy="0.00326077" izz="0.00277255" ixy="0.0" ixz="0.0" iyz="0.0" />
</inertial>
```

## Segment 6

![segment_6](img/segment_6.png)

### Characteristics

| Bounding box| Frame of Reference | Geometric Shape | Center of Mass | V Blender | V MeshLab | V Simple | Mass |
| -------- | ------- | ------- | -------- | ------- | ------- | ------- | ------- |
| x = 0.269   |   Same as joint  | cuboid | $y_{center} = \frac{0.13}{2} \approx 0.071$* | $0.0004m^3$ | $0.001429m^3$  | $0.0032m^3$ | $1.5kg$ |
| y = 0.13 | | | | | | | |
| z = 0.093 | | | | | | | |

* due to odd shape

### Inertia Tensor

In [16]:
mass = 1.5; x = 0.269; y = 0.13; z = 0.093
get_inertia_tensor_of_cuboid(mass, x, y, z) 

array([[0.00319363, 0.        , 0.        ],
       [0.        , 0.01012625, 0.        ],
       [0.        , 0.        , 0.01115763]])

```inertial``` attributes for segment 6:
```
<inertial>
 <mass value="1.5" />
 <origin rpy="0.0 0.0 0.0" xyz="0.0 0.071 0.0" />
 <inertia ixx="0.00319363" iyy="0.01012625" izz="0.01115763" ixy="0.0" ixz="0.0" iyz="0.0" />
</inertial>
```

## Segment 7

![segment_7](img/segment_7.png)

### Characteristics

| Bounding box| Frame of Reference | Geometric Shape | Center of Mass | V Blender | V MeshLab | V Simple | Mass |
| -------- | ------- | ------- | -------- | ------- | ------- | ------- | ------- |
| x = 0.077 | Same as joint | cuboid | Principal Axes | $0.0002m^3$ | $0.000186m^3$ | $0.0002m^3$ | $0.2kg$ |
| y = 0.092 | | | | | | | |
| z = 0.036 | | | | | | | |

### Inertia Tensor

In [17]:
mass = 0.5; x = 0.077; y = 0.092; z = 0.036
get_inertia_tensor_of_cuboid(mass, x, y, z) 

array([[0.00040667, 0.        , 0.        ],
       [0.        , 0.00030104, 0.        ],
       [0.        , 0.        , 0.00059971]])

```inertial``` attributes for segment 7:
```
<inertial>
 <mass value="0.2" />
 <origin rpy="0.0 0.0 0.0" xyz="0.0 0.0 0.0" />
 <inertia ixx="0.00040667" iyy="0.00030104" izz="0.00059971" ixy="0.0" ixz="0.0" iyz="0.0" />
</inertial>
```

## Finger 1 / 2

![finger](img/finger.png)

### Characteristics

| Bounding box| Frame of Reference | Geometric Shape | Center of Mass | V Blender | V MeshLab | V Simple | Mass |
| -------- | ------- | ------- | -------- | ------- | ------- | ------- | ------- |
| x = 0.034 | Same as joint | cuboid | Principal Axes | $0.00003m^3$ | $0.000052m^3$ | - | $0.05kg$ |
| y = 0.056 | | | | | | | |
| z = 0.072 | | | | | | | |



### Inertia Tensor

In [18]:
mass = 0.05; x = 0.034; y = 0.056; z = 0.072
get_inertia_tensor_of_cuboid(mass, x, y, z) 

array([[3.46666667e-05, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.64166667e-05, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.78833333e-05]])

```inertial``` attributes for segment 7:
```
<inertial>
 <mass value="0.05" />
 <origin rpy="0.0 0.0 0.0" xyz="0.0 0.0 0.0" />
 <inertia ixx="3.46666667e-05" iyy="2.64166667e-05" izz="1.78833333e-05" ixy="0.0" ixz="0.0" iyz="0.0" />
</inertial>
```

## Axis

![axis](img/axis.png)

### Characteristics

| Bounding box| Frame of Reference | Geometric Shape | Center of Mass | V Blender | V MeshLab | V Simple | Mass |
| -------- | ------- | ------- | -------- | ------- | ------- | ------- | ------- |
| x = 0.126 | Same as joint | cuboid | $z_{center} = \frac{1.2}{2} = 0.6$ | $0.0086m^3$ | $0.011986m^3$  | $0.009m^3$ | $8.6kg$ |
| y = 0.061 | | | | | | | |
| z = 1.2   | | | | | | | |


### Inertia Tensor

In [19]:
mass = 8.6; x = 0.126; y = 0.061; z = 1.2
get_inertia_tensor_of_cuboid(mass, x, y, z) 

array([[1.03466672, 0.        , 0.        ],
       [0.        , 1.0433778 , 0.        ],
       [0.        , 0.        , 0.01404452]])

```inertial``` attributes for axis:
```
<inertial>
 <mass value="8.6" />
 <origin rpy="0.0 0.0 0.0" xyz="0.0 0.0 0.6" />
 <inertia ixx="1.03466672" iyy="1.0433778" izz="0.01404452" ixy="0.0" ixz="0.0" iyz="0.0" />
</inertial>
```

## Carriage

![carriage](img/carriage.png)

### Characteristics

| Bounding box| Frame of Reference | Geometric Shape | Center of Mass | V Blender | V MeshLab | V Simple | Mass |
| -------- | ------- | ------- | -------- | ------- | ------- | ------- | ------- |
| x = 0.151 | Same as joint | cuboid | $z_{center} = \frac{22}{2} = 0.11$ | $0.0007m^3$ | - | $0.0009m^3$ | $0.7kg$ |
| y = 0.0297 | | | | | | | |
| z = 0.22   | | | | | | | |


### Inertia Tensor

In [20]:
mass = 0.7; x = 0.151; y = 0.0297; z = 0.22
get_inertia_tensor_of_cuboid(mass, x, y, z) 

array([[0.00287479, 0.        , 0.        ],
       [0.        , 0.00415339, 0.        ],
       [0.        , 0.        , 0.00138151]])

```inertial``` attributes for carriage:
```
<inertial>
 <mass value="0.7" />
 <origin rpy="0.0 0.0 0.0" xyz="0.0 0.0 0.11" />
 <inertia ixx="0.00287479" iyy="0.00415339" izz="0.00138151" ixy="0.0" ixz="0.0" iyz="0.0" />
</inertial>
```

## Result

The resulting graphical interpretations of inertia (left) and mass (right) are shown in the following. Some are obviously too conservative and their refinement will be subject to further adaptions.

 <img src="img/robot_inertia.png" alt="robot_inertia" width="300"/> <img src="img/robot_center_of_mass.png" alt="robot_center_of_mass" width="300"/>


# Collision Modelling

Collisions define boundaries of robot elements. They allow to identify physical collisions in case of faulty joint configurations and can be either defined with simple geometric shapes or with meshes.

Our initial approach was to use the same meshes as for the visual tags. This was unfortunately not the optimal approach due to the high resolution of the meshes. According to the [ROS Wiki](http://wiki.ros.org/urdf/XML/link) one should use meshes with less than 1000 faces for the collision tag while one face represents the area between the vertices of a mesh:

![mesh_faces](img/mesh_faces.png)


Our smallest mesh (segment 7) contained approx. 7.000 faces while segment 3 (a more complex mesh) contained more than 35.000 faces.

Reducing the faces in Blender is rather simple:

```
import mesh
in 3D Viewpoint: select Modifier Properties
Add Modifier -> Decimate -> Select Ratio of 0.01 < x < 0.03
export STL
```

The result for segment 2 is shown in the following (left original mesh with 27.000 faces, right decimated mesh with 270 faces)

![segment_2](img/segment_2.png)  <img src="img/segment_2_reduced.png" alt="segment_2_reduced" width="400"/>
